# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter

In [2]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [3]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [4]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [5]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

13:28:50 | NewsletterAgent.newsletter_agent | INFO | Test info message
13:28:50 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
13:28:50 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [6]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
session_id = 'test_newsletter_20250926123206126884'
step_name = 'step_05_cluster_by_topic'
# del session_id

do_download=False
process_since='2025-09-24 18:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since)
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

13:30:58 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Using provided state with 323 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250926123206126884
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250926123206126884


In [7]:
state.get_status()

{'headlines': {'total': 323},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_06_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 55.55555555555556,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [8]:
state.get_current_step()


'step_06_rate_articles'

In [10]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


13:31:59 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting check_workflow_status
13:31:59 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.46s
📊 Final result:
Current workflow status:
- Progress: 44.4% (4/9 complete)
- Status: 4 complete, 0 started, 0 failed, 5 not started
- Next step: Step 5 — Cluster By Topic

Step-by-step:
- Step 1 (Fetch URLs): complete
- Step 2 (Filter URLs): complete
- Step 3 (Download Articles): complete
- Step 4 (Extract Summaries): complete
- Step 5 (Cluster By Topic): not_started
- Step 6 (Rate Articles): not_started
- Step 7 (Select Sections): not_started
- Step 8 (Draft Sections): not_started
- Step 9 (Finalize Newsletter): not_started

Data summary:
- Total articles: 323
- AI-related: 323
- Clusters: 0
- Sections: 0

Would you like me to run Step 5 (Cluster By Topic) now, run a different step, or do something else?


In [9]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


12:32:18 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting check_workflow_status
12:32:18 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed check_workflow_status
12:32:19 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting Step 1: Gather URLs
2025-09-26 12:32:19,327 - fetcher_4583246608 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-26 12:32:19,343 - fetcher_4583246608 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-26 12:32:19,344 - fetcher_4583246608 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-26 12:32:19,345 - fetcher_4583246608 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-26 12:32:19,345 - fetcher_4583246608 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-26 12:32:19,345 - fetcher_4583246608 - DEBU

▶ Starting Step 1: step_01_fetch_urls


2025-09-26 12:32:19,532 - fetcher_4583246608 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-09-26 12:32:19,532 - fetcher_4583246608 - INFO - [fetch_html] HTML fetch successful for Reddit: 62 articles
2025-09-26 12:32:19,532 - fetcher_4583246608 - INFO - [fetch_rss] Fetching RSS from Techmeme: https://www.techmeme.com/feed.xml
2025-09-26 12:32:19,533 - fetcher_4583246608 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-09-26 12:32:19,533 - fetcher_4583246608 - INFO - [fetch_html] Using existing HTML file from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-09-26 12:32:19,533 - fetcher_4583246608 - INFO - [fetch_html] Parsing HTML file: download/sources/The_Verge.html
2025-09-26 12:32:19,665 - fetcher_4583246608 - INFO - [fetch_html] Parsed HTML file: download/sources/The_Verge.html
2025-09-26 12:32:19,665 - fetcher_4583246608 - INFO - [fetch_html] HTML fetch successful for T

,source,url
0,Ars Technica,20
1,Bloomberg,25
2,Business Insider,17
3,FT,48
4,Feedly AI,73
5,Hacker News,30
6,HackerNoon,50
7,New York Times,25
8,NewsAPI,95
9,Reddit,53


,source,title,url,published,rss_summary,id
0,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,0
1,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,1
2,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,2
3,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,3
4,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,4
...,...,...,...,...,...,...
608,NewsAPI,Google’s Data Commons MCP Server Anchors AI in...,http://www.pymnts.com/news/artificial-intellig...,2025-09-25T16:00:29Z,NaN,608
609,NewsAPI,Medicare to Put AI in Charge of Prior Authoriz...,http://www.pymnts.com/artificial-intelligence-...,2025-09-25T13:00:43Z,NaN,609
610,NewsAPI,iPhone 17: Hidden Features Apple Didn’t Tell Y...,https://www.geeky-gadgets.com/iphone-17-hidden...,2025-09-25T15:00:05Z,NaN,610
611,NewsAPI,Build a Second Brain with AI in Just 10 Minute...,https://www.geeky-gadgets.com/build-an-ai-seco...,2025-09-25T13:19:21Z,NaN,611


12:32:20 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed Step 1: Gathered 695 articles
12:32:22 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting check_workflow_status
12:32:22 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed check_workflow_status


⏱️  Total execution time: 14.91s
📊 Final result:
Step 1 completed successfully.

Summary:
- Step 1 (Fetch URLs): complete
- Articles gathered: 695 discovered from 17 sources (RSS); 613 articles stored in persistent state
- Progress: 11.1% (1/9 complete)
- Next step: Step 2 — Filter URLs (AI-related)

Would you like me to continue and run Step 2 now, or pause here?


In [10]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,0
1,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,1
2,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,2
3,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,3
4,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,4
...,...,...,...,...,...,...
608,NewsAPI,Google’s Data Commons MCP Server Anchors AI in...,http://www.pymnts.com/news/artificial-intellig...,2025-09-25T16:00:29Z,NaN,608
609,NewsAPI,Medicare to Put AI in Charge of Prior Authoriz...,http://www.pymnts.com/artificial-intelligence-...,2025-09-25T13:00:43Z,NaN,609
610,NewsAPI,iPhone 17: Hidden Features Apple Didn’t Tell Y...,https://www.geeky-gadgets.com/iphone-17-hidden...,2025-09-25T15:00:05Z,NaN,610
611,NewsAPI,Build a Second Brain with AI in Just 10 Minute...,https://www.geeky-gadgets.com/build-an-ai-seco...,2025-09-25T13:19:21Z,NaN,611


In [11]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
8,NewsAPI,95
4,Feedly AI,73
9,Reddit,53
6,HackerNoon,50
12,The Register,50
3,FT,48
13,The Verge,30
5,Hacker News,30
16,Washington Post,28
1,Bloomberg,25


In [12]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


12:32:27 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting Step 2: Filter URLs
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=458, user_len=954


▶ Starting Step 2: step_02_filter_urls


12:40:58 | NewsletterAgent.test_newsletter_20250926123206126884 | ERROR | Invalid data in filter_dataframe_chunk: Invalid \uXXXX escape: line 1 column 67533 (char 67532)
12:41:11 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed Step 2: 339 AI-related articles


⏱️  Total execution time: 524.37s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Filtered 613 headlines to 339 AI-related articles.

📊 Results stored in persistent state. Current step: step_03_download_articles


In [13]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

12:41:11 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting Step 3: Download Articles
12:41:11 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting concurrent scraping of 339 AI-related articles


▶ Starting Step 3: step_03_download_articles


12:41:12 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Launching browser for 339 URLs with 16 concurrent workers
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 fetching 1 of 339 https://www.bloomberg.com/news/articles/2025-09-24/openai-pairs-with-sap-to-bring-chatgpt-to-german-government
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Skipping ignored domain: www.bloomberg.com
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 completed https://www.bloomberg.com/news/articles/2025-09-24/openai-pairs-with-sap-to-bring-chatgpt-to-german-government with status: success
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 fetching 2 of 339 https://venturebeat.com/ai/ficos-answer-to-ai-risk-a-foundation-model-that-scores-every-output-for
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | File already exists: download/html/FICO_s_answ

12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | File already exists: download/html/AI_gone_rogue__Models_may_try_to_stop_people_from_shutting_them_down__Google_warns.html
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 3 completed https://go.theregister.com/feed/www.theregister.com/2025/09/22/google_ai_misalignment_risk/ with status: success
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 3 fetching 15 of 339 https://go.theregister.com/feed/www.theregister.com/2025/09/25/oracle_18_billion_debt/
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | File already exists: download/html/Oracle_saddles_up_with__18B_debt_amid_AI_infrastructure_gamble.html
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 3 completed https://go.theregister.com/feed/www.theregister.com/2025/09/25/oracle_18_billion_debt/ with status: success
12:41:13 | NewsletterAgent.test_new

12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 5 fetching 28 of 339 https://biztoc.com/x/98d986467b56bf54
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Rate limiting domain biztoc.com, will retry later (need to wait 2.0s)
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 5 re-queued rate-limited URL: https://biztoc.com/x/98d986467b56bf54
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 6 fetching 28 of 339 https://www.bloomberg.com/news/articles/2025-09-24/alibaba-integrates-nvidia-robotics-software-in-its-ai-platform
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Skipping ignored domain: www.bloomberg.com
12:41:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 6 completed https://www.bloomberg.com/news/articles/2025-09-24/alibaba-integrates-nvidia-robotics-software-in-its-ai-platform with status: success
12:41:13 | N

...

12:51:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Saving HTML to download/html/Elon_Musk_s_xAI_accuses_OpenAI_of_stealing_trade_secrets_in_new_lawsuit.html
12:51:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 completed https://www.theguardian.com/technology/2025/sep/25/elon-musk-xai-openai-sam-altman-lawsuit with status: success
12:51:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 fetching 338 of 339 https://www.washingtonpost.com/technology/2025/09/16/deepseek-ai-security/
12:51:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | File already exists: download/html/AI_firm_DeepSeek_writes_less-secure_code_for_groups_China_disfavorsResearch_by_a_U.S._security_firm_points_to_the_country_s_leading_player_in_AI_providing_higher-quality_results_for_some_purposes_than_others..html
12:51:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 0 completed https://www.washingtonpost

12:52:49 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Saving HTML to download/html/Improved_Gemini_2.5_Flash_and_Flash-Lite.html
12:52:49 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Worker 2 completed https://developers.googleblog.com/en/continuing-to-bring-you-our-latest-models-with-an-improved-gemini-2-5-flash-and-flash-lite-release/ with status: success
12:52:49 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Closing browser
12:52:52 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed scraping 339 URLs: 339 successful, 0 failed
12:52:54 | NewsletterAgent.test_newsletter_20250926123206126884 | WARNING | Error extracting og:description: 'NoneType' object has no attribute 'strip'
12:53:14 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Updated 338 URL records with final URLs and AI classification


Starting with 338 rows...
Processing 338 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

12:53:23 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed Step 3: Downloaded 321 articles


Filtering dataframe...
  Pair: 310 vs 309
    310: NewsAPI - Liquid AI debuts extremely small, high-performance foundation models for on-device processing
    309: NewsAPI - Liquid AI debuts extremely small, high-performance foundation models for on device
  Pair: 73 vs 323
    73: Feedly AI - Fermi America IPO: Stock listing date nears for AI data center and power company backed by Rick Perry
    323: NewsAPI - Fermi America IPO: Stock listing date nears for AI data center and power company backed by Rick Perry
  Pair: 83 vs 184
    83: Feedly AI - Oracle saddles up with $18B debt amid AI infrastructure gamble
    184: The Register - Oracle saddles up with $18B debt amid AI infrastructure gamble
  Pair: 327 vs 336
    327: NewsAPI - Medicare to Put AI in Charge of Prior Authorization With 2026 Pilot Program
    336: NewsAPI - Medicare to Put AI in Charge of Prior Authorization With 2026 Pilot Program
  Pair: 327 vs 77
    327: NewsAPI - Medicare to Put AI in Charge of Prior Authorizat

In [14]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

12:53:23 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Processing 323 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1204, user_len=43


▶ Starting Step 4: step_04_extract_summaries
⏱️  Total execution time: 23.98s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 323/323 articles.
💾 Summaries stored in headline DataFrame.


In [11]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Using model 'gpt-4.1-mini' for topic extraction
13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Processing 323 articles for topic extraction
13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Initialized LLMagent:
system_prompt: 
# Role and Objective
You are an expert AI news analyst. Your task is to extract a list of up to **5** distinct topics from provided news summaries (or an empty list if no topics can be extracted).

# Input Format
You will receive a list of news summary objects in JSON format including fields "id" and "summary".

# Outp

13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":201,
    "summary":"- Google is rolling out a significant AI feature expansion in Chrome, integrating its Gemini AI to enhance browsing through a desktop Gemini button that provides summaries and answers related to open tabs and integrates with apps like Calendar and YouTube.\n- AI Mode is increasingly integrated into Chrome's search experience, allowing users to start AI-powered searches directly from the omnibox and interact with content through AI-driven question prompts and follow-ups, signaling a possible shift towards AI as the default search mode.\n- Chrome's security features are enhanced by AI, with Gemini detecting scams and fake alerts, plus an upgraded password manager that can identify compromised passwords and auto-update them, although this new functionality carries potential risks."
  },
  {
    "id":21

13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":272,
    "summary":"- Nvidia is investing $100 billion in building 'gigantic AI factories' to enhance processing power for AI models like ChatGPT.\n- This strategic investment aims to maintain Nvidia's leading position in the AI chip market amid growing demand for AI technologies.\n- The move is expected to drive significant market growth and solidify Nvidia's role at the core of future AI developments and commercial applications."
  },
  {
    "id":263,
    "summary":"- Modular, a three-year-old start-up, has raised $250 million to develop an alternative to Nvidia's Cuda software, aiming to challenge Nvidia's dominance in AI chip software.\n- The funding highlights significant market interest in diversifying AI development tools and reducing dependency on Nvidia's proprietary ecosystem.\n- Modular's technology could i

13:32:26 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":151,
    "summary":"- Elon Musk's xAI secured an agreement to provide its AI chatbot Grok to US federal agencies at $0.42 per organization until March 2027, under the General Services Administration's OneGov initiative.\n- This pricing is lower and the contract longer than any previous OneGov deals, positioning Grok competitively against rivals like OpenAI's ChatGPT; xAI will also supply engineers to assist federal adoption.\n- The move supports government modernization efforts for AI deployment but faces criticism from Democrats and advocacy groups over Grok's accuracy and bias concerns."
  },
  {
    "id":286,
    "summary":"- Micron Technology's stock price doubled within four months following a previous upgrade from \"Hold\" to \"Buy,\" driven largely by AI-related hype and strong market performance.\n- Recent rapi

▶ Starting Step 5: step_05_cluster_by_topic


...

13:34:33 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Result: results_list=[TopicExtraction(id=147, topics_list=['Reinforcement Learning', 'Language Models', 'Multilingual Models', 'Mathematical Reasoning', 'Coding Assistants', 'China', 'Gen AI']), TopicExtraction(id=128, topics_list=['AI Accelerators', 'Huawei', 'Semiconductor Chips', 'Hardware', 'Infrastructure', 'China', 'Economics']), TopicExtraction(id=130, topics_list=['AI Hardware Ban', 'Nvidia', 'China', 'Chinese Tech Policy', 'Semiconductor Chips', 'Policy And Regulation', 'Economics']), TopicExtraction(id=79, topics_list=['Defense Contracts', 'Scale AI', 'National Security', 'Gen AI', 'Computer Vision', 'Cybersecurity', 'Deals']), TopicExtraction(id=75, topics_list=['AMD ROCm', 'GPU Software', 'High Performance Computing', 'Hardware', 'Gen AI', 'Semiconductor Chips', 'Open Source']), TopicExtraction(id=96, topics_list=['AI Spending', 'Gen AI', 'Economics', 'Infrastructure', 'GenAI Integration', 'Consumer El

13:34:55 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Result: results_list=[TopicExtraction(id=191, topics_list=['ChatGPT Features', 'Personalized Updates', 'Mobile Applications', 'Virtual Assistants', 'Privacy', 'Retrieval Augmented Generation']), TopicExtraction(id=182, topics_list=['AI Stock Picking', 'Retail Investors', 'Investment Analysis', 'Fintech', 'Finance', 'Stocks']), TopicExtraction(id=202, topics_list=['Gemini Robotics', 'Robotic AI Models', 'Autonomous Robots', 'Agents', 'Computer Vision', 'Language Models']), TopicExtraction(id=206, topics_list=['RoboBallet', 'Manufacturing Robots', 'Robot Coordination', 'Manufacturing', 'Robots', 'Job Automation']), TopicExtraction(id=199, topics_list=['Stargate Project', 'AI Data Centers', 'Infrastructure Expansion', 'Energy', 'Jobs And Careers', 'Hardware', 'Economics']), TopicExtraction(id=183, topics_list=['Persian Taarof', 'Cultural AI Bias', 'Testing', 'Language Models', 'Bias And Fairness', 'Society And Cultur

13:35:01 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Result: results_list=[TopicExtraction(id=30, topics_list=['Life Sciences', 'AI Agents', 'Healthcare Integration', 'Products', 'Governance', 'Privacy', 'Customer Service']), TopicExtraction(id=63, topics_list=['Autonomous AI', 'Data Engineering', 'Agents', 'Funding', 'Venture Capital', 'Cloud Platforms', 'Testing']), TopicExtraction(id=25, topics_list=['AI Infrastructure', 'Data Management', 'Hybrid Cloud', 'Products', 'Governance', 'Open Source', 'Cybersecurity']), TopicExtraction(id=152, topics_list=['AI Hardware', 'Energy Efficiency', 'Data Centers', 'Funding', 'Korea', 'Semiconductor Chips', 'Infrastructure']), TopicExtraction(id=300, topics_list=['Web Security', 'Cloudflare', 'Cybersecurity', 'Privacy', 'Legal Issues']), TopicExtraction(id=303, topics_list=[]), TopicExtraction(id=70, topics_list=['Music AI', 'Content Moderation', 'Spam Filters', 'Intellectual Property', 'Deepfakes', 'Spotify']), TopicExtractio

13:35:11 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Result: results_list=[TopicExtraction(id=139, topics_list=['Critical Thinking', 'AI Automation', 'Workforce Skills', 'Jobs And Careers', 'Amazon Web Services', 'Education']), TopicExtraction(id=146, topics_list=['CoreWeave', 'OpenAI', 'AI Infrastructure', 'Investments', 'UK', 'Venture Capital', 'Stocks']), TopicExtraction(id=72, topics_list=['Full Self-Driving', 'Reinforcement Learning', 'Tesla', 'Autonomous Vehicles', 'Safety And Alignment', 'Neural Networks', 'Policy And Regulation']), TopicExtraction(id=101, topics_list=['Stock Valuation', 'Tesla', 'Robotaxis', 'Autonomous Vehicles', 'Optimus Robots', 'Energy Storage', 'Finance']), TopicExtraction(id=209, topics_list=['Alibaba', 'AI Investment', 'Nvidia', 'Data Centers', 'Stocks', 'Global Expansion', 'Infrastructure']), TopicExtraction(id=112, topics_list=['AI Governance', 'Policy And Regulation', 'Lethal Autonomous Weapons', 'Information Integrity', 'Security 

⏱️  Total execution time: 165.02s
📊 Final result:
❌ Step 5 failed: "['tags'] not found in axis"


In [28]:
headline_df

,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,last_updated,text_path,content_length,summary,description,topics,extended_summary,cluster,cluster_name
0,191,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,True,success,https://arstechnica.com/ai/2025/09/chatgpt-pul...,download/html/ChatGPT_Pulse_delivers_morning_u...,2025-09-25T20:30:52Z,download/text/ChatGPT_Pulse_delivers_morning_u...,2521,"- OpenAI launched ChatGPT Pulse, a new mobile ...",New mobile chatbot feature analyzes conversati...,"[ChatGPT Pulse, Personalization, Asynchronous ...",ChatGPT Pulse delivers morning updates based o...,-1,Other
1,182,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,True,success,https://arstechnica.com/information-technology...,download/html/Experts_urge_caution_about_using...,2025-09-25T18:10:50Z,download/text/Experts_urge_caution_about_using...,2611,- At least 13% of retail investors use AI tool...,AI-selected portfolios might perform well in a...,"[Retail Investors, Investment Tools, AI in Fin...",Experts urge caution about using ChatGPT to pi...,2,AI Chatbots for Stock Picking
2,202,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,True,success,https://arstechnica.com/google/2025/09/google-...,download/html/Google_DeepMind_unveils_its_firs...,2025-09-25T16:00:59Z,download/text/Google_DeepMind_unveils_its_firs...,2435,- Google DeepMind introduced Gemini Robotics 1...,DeepMind researchers believe this is the dawn ...,"[DeepMind, Robotics, AI Models, Agents, Comput...",Google DeepMind unveils its first “thinking” r...,-1,Other
3,206,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,True,success,https://arstechnica.com/science/2025/09/deepmi...,download/html/DeepMind_s_robotic_ballet__An_AI...,None,download/text/DeepMind_s_robotic_ballet__An_AI...,2346,"- DeepMind developed RoboBallet, an AI system ...",An AI figures out how robots can get jobs done...,"[DeepMind, Manufacturing Robots, AI Coordinati...",DeepMind’s robotic ballet: An AI for coordinat...,-1,Other
4,199,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,True,success,https://arstechnica.com/ai/2025/09/why-does-op...,download/html/Why_does_OpenAI_need_six_giant_d...,None,download/text/Why_does_OpenAI_need_six_giant_d...,2461,"- OpenAI, Oracle, and SoftBank are developing ...",OpenAI’s new $400 billion announcement reveals...,"[Stargate Project, AI Data Centers, AI Infrast...",Why does OpenAI need six giant data centers?\n...,27,OpenAI's Large-Scale AI Data Centers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,229,NewsAPI,From 4% To 20%: Women's Enrolment Goes Up Four...,https://www.ndtvprofit.com/technology/from-4-t...,2025-09-25T13:19:16Z,None,True,success,https://www.ndtvprofit.com/technology/from-4-t...,download/html/From_4__To_20___Women_s_Enrolmen...,2025-09-25T13:19:16Z,download/text/From_4__To_20___Women_s_Enrolmen...,2004,- Women's enrolment in AI and machine learning...,Up to 70% of women candidates who enrolled in ...,"[Women in Tech, AI Education, India, STEM Enro...",From 4% To 20%: Women's Enrolment Goes Up Four...,-1,Other
319,22,NewsAPI,I met Sam Altman in Texas. He’s turning the ra...,https://finance.yahoo.com/news/met-sam-altman-...,2025-09-25T16:29:49Z,None,True,success,https://

In [12]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


13:35:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Starting check_workflow_status
13:35:13 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.25s
📊 Final result:
Current workflow status:
- Progress: 44.4% (4/9 complete)
- Status: 4 complete, 0 started, 1 failed, 4 not started
- Next step: Step 5 — Cluster By Topic (failed)

Step-by-step:
- Step 1 (Fetch URLs): complete
- Step 2 (Filter URLs): complete
- Step 3 (Download Articles): complete
- Step 4 (Extract Summaries): complete
- Step 5 (Cluster By Topic): error — "['tags'] not found in axis"
- Step 6 (Rate Articles): not_started
- Step 7 (Select Sections): not_started
- Step 8 (Draft Sections): not_started
- Step 9 (Finalize Newsletter): not_started

Data summary:
- Total articles: 323
- AI-related: 323
- Clusters: 0
- Sections: 0

Action needed:
- The clustering step failed with an error indicating missing 'tags' in the data axis. I can inspect the detailed persistent state to debug and fix the issue, then retry Step 5. Would you like me to inspect the state now and attempt to re-run Step 5?


In [9]:
from newsletter_state import StepStatus
agent.state.step_05_cluster_by_topic = StepStatus.NOT_STARTED


- rate articles
- load sources into db with reputation
- get domain from each url and put in headline_df
- look up source and reputation
- prompt for on topic , important, high quality
- run bradley terry
- combine ratings

    aidf['rating'] = aidf['reputation'] \
        + aidf['adjusted_len'] \
        + aidf['on_topic'] \
        + aidf['importance'] \
        - aidf['low_quality'] \
        + aidf['bt_z'] \
        + aidf['recency_score']

set selected flag using rating
store to db

- next steps select sections
- clean clusters , combine clusters, select sections 

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
state = await agent.get_state_direct()


In [ ]:
print(status_result)
